# Merge dataframes and create columns

In [3]:
import pandas as pd
import numpy as np
import os

year=2023
# Load data
current_folder = globals()['_dh'][0]
minute_prices = pd.read_csv(os.path.join(current_folder, './{}/minute_prices.csv'.format(year)))
imbalance_prices = pd.read_csv(os.path.join(current_folder, './{}/imbalance_prices.csv'.format(year)))

minute_prices= minute_prices.replace({',': '.'}, regex=True)
imbalance_prices= imbalance_prices.replace({',': '.'}, regex=True)

minute_prices = minute_prices[['datum', 'tijd', 'Hoogste_prijs_opregelen', 'Mid_prijs_opregelen', 'Laagste_prijs_afregelen']]
minute_prices = minute_prices.rename(columns={'datum': 'date', 'tijd': 'time', 'Hoogste_prijs_opregelen': 'high_feed_price', 'Mid_prijs_opregelen': 'mid_price', 'Laagste_prijs_afregelen': 'low_take_price'})

minute_prices['high_feed_price'] = pd.to_numeric(minute_prices['high_feed_price'])
minute_prices['mid_price'] = pd.to_numeric(minute_prices['mid_price'])
minute_prices['low_take_price'] = pd.to_numeric(minute_prices['low_take_price'])

imbalance_prices['Afnemen'] = pd.to_numeric(imbalance_prices['Afnemen'])
imbalance_prices['invoeden'] = pd.to_numeric(imbalance_prices['invoeden'])
imbalance_prices['regeltoestand'] = pd.to_numeric(imbalance_prices['regeltoestand'])

print(minute_prices.shape)
print(imbalance_prices.shape)
print(minute_prices['date'][0])


minute_prices.insert(2, "month", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(3, "day_of_week", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(4, "hour_of_day", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(5, "imbalance_take_price", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(6, "imbalance_feed_price", np.zeros(minute_prices.shape[0]), True)
minute_prices.insert(7, "imbalance_regulation_state", np.zeros(minute_prices.shape[0]), True)

# Merge dataframes
j = -1
for i in range (minute_prices.shape[0]):
  if i % 15 == 0:
    j+=1
    if not pd.isnull(minute_prices['mid_price'][i]):
      mid = minute_prices['mid_price'][i]
    else:
      mid = imbalance_prices['invoeden'][j]

  afnemen = imbalance_prices['Afnemen'][j]
  invoeden = imbalance_prices['invoeden'][j]
  regeltoestand = imbalance_prices['regeltoestand'][j]

  mid_price = mid

  minute_prices.loc[i, 'imbalance_take_price'] = afnemen
  minute_prices.loc[i, 'mid_price'] = mid_price
  minute_prices.loc[i, 'imbalance_feed_price'] = invoeden
  minute_prices.loc[i, 'imbalance_regulation_state'] = regeltoestand


# Create date columns
minute_prices['date'] = pd.to_datetime(minute_prices['date'], format='%d-%m-%Y')
minute_prices['time'] = pd.to_datetime(minute_prices['time'], format='%H:%M')

minute_prices['month'] = minute_prices['date'].dt.month
minute_prices['day_of_week'] = minute_prices['date'].dt.dayofweek
minute_prices['hour_of_day'] = minute_prices['time'].dt.hour

minute_prices['time'] = minute_prices['time'].dt.time

df = minute_prices

print(df.head(5))

df.to_csv(os.path.join(current_folder, './{}/test_imb.csv'.format(year)), index=False)

(525600, 5)
(35040, 12)
01-01-2023
        date      time  month  day_of_week  hour_of_day  imbalance_take_price  \
0 2023-01-01  00:00:00      1            6            0                -209.4   
1 2023-01-01  00:01:00      1            6            0                -209.4   
2 2023-01-01  00:02:00      1            6            0                -209.4   
3 2023-01-01  00:03:00      1            6            0                -209.4   
4 2023-01-01  00:04:00      1            6            0                -209.4   

   imbalance_feed_price  imbalance_regulation_state  high_feed_price  \
0                -209.4                        -1.0              NaN   
1                -209.4                        -1.0              NaN   
2                -209.4                        -1.0              NaN   
3                -209.4                        -1.0              NaN   
4                -209.4                        -1.0              NaN   

   mid_price  low_take_price  
0      12.31  

# Train dataset

In [5]:
# Shift the data and remove the first 2 minutes

df = pd.read_csv('./{}/test_imb.csv'.format(year))

# Add two rows
df.loc[-1] = df.iloc[0]
df.loc[-2] = df.iloc[0]
df.index = df.index + 2
df.sort_index(inplace=True)

# Remove the feed and take price from the last two minutes
# This is done because the Tennet data has 2 minutes delay and so we don't have the info for the last 2 minutes in time
for i in range(len(df)):
    if i % 15 == 0:
        df.loc[i, 'high_feed_price'] = None
        df.loc[i, 'low_take_price'] = None
        df.loc[i + 1, 'high_feed_price'] = None
        df.loc[i + 1, 'low_take_price'] = None

df = df.iloc[2:]
df.to_csv('./{}/train_imb.csv'.format(year), index=False)

# Normalize data on mid price

In [ ]:
import matplotlib.pyplot as plt
# Plotting frequency histogram
plt.hist(df['low_feed_price'], bins=len(df['low_feed_price'].unique()), edgecolor='black')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Frequency Histogram')
plt.grid(True)
plt.show()